In [ ]:
import random
import mysql.connector
import json
import string
import datetime as dt
from pymongo import MongoClient

def getGenero(argument):
    switcher = {        
        5: "action",
        6: "adventure",
        7: "casual",
        8: "strategy",
        9: "RPG",
        10: "simulation",
        11: "early_access",
        12: "free_to_play",
        13: "sports",
        14: "racing",
        15: "massively_multiplayer"
    }
    return switcher.get(argument, argument)

myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]
colmul = mydb["multidimensional"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)
from datetime import datetime,timedelta
hoy = datetime.now()
dias = timedelta(days=7)
semana_pasada = hoy-dias  
#se comprueba que sea domingo (0)
comp=hoy.strftime("%w")
print("Día de la semana:",comp)
if (comp == '0') :#--------------------------------------------------------------cuidado
    esjuego=False
    esDLC=False
    hoy=hoy.strftime("%Y-%m-%d")
    #semana_pasada=semana_pasada.strftime("%Y:%m:%d")
    
    semana_pasada ="2017-01-01"#-------------------------------------------------para pruebas
    print(hoy,semana_pasada)
    mycursor = mydb.cursor(buffered=True)

    mycursor.execute("SELECT * FROM transacciones_compra WHERE fecha_compra BETWEEN %s AND  %s", (semana_pasada,hoy,))
    transacciones = mycursor.fetchall()
    tipo_movimiento = "compra"
    for t in transacciones :
   
    #"movimiento"
        precio_final = t[1]
        mycursor.execute("SELECT * FROM compra_juegos WHERE id_transaccion = %s", (t[0],))
        compra = mycursor.fetchall()
        cantidad = len(compra)
    #fecha
        anyo=t[2].strftime("%Y")
        mes=t[2].strftime("%m")
        dia=t[2].strftime("%d")
    #juego
        if(len(compra)>0) :
            esjuego=True
            esDLC=False
            mycursor.execute("SELECT id_juego FROM claves_juegos WHERE id_clave = %s", (compra[0][2],))
            puntero_juego = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM juegos WHERE id_juego = %s", (puntero_juego,))
            juego = mycursor.fetchone()
            genero= []
            #sacar genero
            for a in range (5,15) :
                if (juego[a]==True) :
                    genero.append(getGenero(a))
            precio_juego=juego[3]
            clasificacion=juego[1]
     #dlc
        if(len(compra)<=0) :
            print("***********************ha entrado en dlc***********************************")
            esDLC=True
            esjuego=False
            mycursor.execute("SELECT * FROM compra_dlcs WHERE id_transaccion = %s", (t[0],))
            compra = mycursor.fetchall()
            cantidad = len(compra)
            mycursor.execute("SELECT id_dlc FROM claves_dlc WHERE id_clave = %s", (compra[0][2],))
            puntero_dlc = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM dlcs WHERE id_dlc = %s", (puntero_dlc,))
            dlc = mycursor.fetchone()
            tipo= dlc[3]
            precio_dlc=dlc[1]
            
    #proveedor
        puntero_proveedor=compra[0][0]
        mycursor.execute("SELECT * FROM proveedor WHERE id_proveedor = %s", (puntero_proveedor,))
        provedor = mycursor.fetchone()
        pais= provedor[3]
        valoracion=provedor[1]
        
    #insertar datos en multidimensional    
        if esjuego:
            datos = {
                "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
                "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
                "juego":{"puntero_juego":puntero_juego,"genero":genero,"precio_juego":precio_juego,"clasificacion":clasificacion},
                "proveedor":{"puntero_proveedor":puntero_proveedor,"pais":pais,"valoracion":valoracion}
             }
        elif esDLC:
             datos = {
                "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
                "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
                "dlc":{"puntero_dlc":puntero_dlc,"tipo":tipo,"precio_dlc":precio_dlc},
                "proveedor":{"puntero_proveedor":puntero_proveedor,"pais":pais,"valoracion":valoracion}
             }
        print("---------------------------------------------------------------")        
        print(datos)
        x = colmul.insert_one(datos) 
        
    